<a href="https://colab.research.google.com/github/code-lover636/Intro_Pytorch/blob/main/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [7]:
class Model(nn.Module):
    # Input layer - 4 features of flower --> Hidden layer 1
    # -->H2 -->output
    def __init__(self, inp_features=4, h1=8, h2=9, out_features=3):
        super().__init__() # instantiate nn.Module
        self.fc1 = nn.Linear(inp_features,h1)
        self.fc2 = nn.Linear(h1, h2)
        self.out = nn.Linear(h2, out_features)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)

        return x


In [8]:
# Manual seed for randomisation
torch.manual_seed(41)
model = Model()

In [10]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
df = pd.read_csv(url)
df

In [13]:
df['variety'] = df['variety'].replace('Setosa',0.0)
df['variety'] = df['variety'].replace('Versicolor',1.0)
df['variety'] = df['variety'].replace('Virginica',2.0)
df

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2.0
146,6.3,2.5,5.0,1.9,2.0
147,6.5,3.0,5.2,2.0,2.0
148,6.2,3.4,5.4,2.3,2.0


In [14]:
# train test and split
X = df.drop('variety', axis=1)
y = df['variety']

X = X.values
y = y.values

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)


In [16]:
# Converting to tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [17]:
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [18]:
# Measure error
criterion = nn.CrossEntropyLoss()
# Adam Optimizer, learning rate (lr)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [19]:
# Epochs - one run through data
epochs = 100
losses = []
for i in range(epochs):
    # forward and predict
    y_pred = model.forward(X_train)

    # Measure error
    loss = criterion(y_pred, y_train)
    losses.append(loss.detach().numpy())
    #print every 10 epoch
    if i % 10 == 0:
        print(f'Epoch {i}: loss = {loss}')
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


Epoch 0: loss = 1.125203251838684
Epoch 10: loss = 1.0097211599349976
Epoch 20: loss = 0.8162348866462708
Epoch 30: loss = 0.585993230342865
Epoch 40: loss = 0.4003390073776245
Epoch 50: loss = 0.2679471969604492
Epoch 60: loss = 0.17963500320911407
Epoch 70: loss = 0.12165628373622894
Epoch 80: loss = 0.08606517314910889
Epoch 90: loss = 0.06522617489099503
